# Converting all kinds of (non-English-language) documents into text

Have a collection of documents? Word docs, HTML files, PDFs, _image-based_ PDFs, and anything else? Don't worry, this notebook has you covered, **especially if your text isn't in English!**

In this case we're going to use Greek, but it works just as well for other languages and characters sets, as well.

<p class="reading-options">
  <a class="btn" href="/text-analysis/processing-documents-with-apache-tika-greek">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/text-analysis/notebooks/Processing documents with Apache Tika (Greek).ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/text-analysis/notebooks/Processing documents with Apache Tika (Greek).ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

## Installation

These installation instructions only work on OS X, but it's possible to get the same software running on Windows.

### Tesseract

[Tesseract](https://github.com/tesseract-ocr/tesseract) is a piece of software that performs OCR, converting images of text into actual text. If we need to perform OCR on more languages than just English, we'll also need to install `tesseract-lang` to add more languages to the mix.
    
```
brew install tesseract tesseract-lang
```

### Tika

[Tika](https://tika.apache.org/) is an incredible piece of software that converts just about any kind of document to text. It requires Java - I installed Java from https://www.java.com/en/download/ and it didn't work, so you'll need to use the install command below.

```
brew cask install adoptopenjdk
brew install tika 
```

Tika will automatically know about tesseract.

### Python bindings for Tika

Tika is a piece of software that exists _outside of Python_. If we want Python to be able to use Tika, we'll need to install the **Python bindings** for TIka.

```
pip install tika
```

If you'd like to just run this all from the notebook, uncomment and run the cell below. **You'll need to type in your password for the `adoptopenjdk` one, so be sure to pay attention to when it asks you.**

In [1]:
# !brew install tesseract tesseract-lang
# !brew cask install adoptopenjdk
# !brew install tika 
# !pip install tika

## Documents we'll be using

* `pdf`: http://www.data.gov.gr/dataset/a7b2fb4b-e6cb-452f-9e5f-4ebbc9292fa8/resource/04d455da-f644-430e-8b2b-57853358fb91/download/2019---.pdf
* `doc`: http://www.data.gov.gr/dataset/3e5cce22-026c-4f3a-9581-95a574f83ef2/resource/0fa049fa-e821-4971-a5b4-9559ffed99a3/download/.docx
* `png` for OCR: https://i.stack.imgur.com/HUy7V.png

## Confirm tesseract works

In [2]:
# Download the image
!curl -O https://i.stack.imgur.com/HUy7V.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  216k  100  216k    0     0   350k      0 --:--:-- --:--:-- --:--:--  350k


![](https://i.stack.imgur.com/HUy7V.png)

In [8]:
!tesseract HUy7V.png stdout -l grc

Η Αϑήνα (Αϑῆναι στὰ ἀρχαία ελληνικά και τὴν καϑαρεύουσα) εἶναι ἡ πρω-
τεύουσα τῆς Βλλάδας. Ἐπίσης είναι ἡ ἔδρα τῆς Περιφέρειας Αττικής. Βρίσχεται
στὴ Στερεά (Κεντρική) Ἑλλάδα και αποτελεί εύρωστο οιχονομιχό, πολιτιστικό χαι
διοικητικό κέντρο τῆς χώρας. Πήρε το ὀνομά τῆς από τὴ ϑεά Αϑηνά που ἦταν χαι
ἡ προστάτιδά της. Η Αϑήνα σήμερα εἰναι μία σύγχρονη πόλη αλλά χαι διάσημη,
χκαϑώς στὴν αρχαιότητα ἦταν πανίσχυρη πόλη-χράτος και σημαντικότατο χέντρο
πολιτισμού. ϑεωρείται ἡ ἱιστορικότερη πόλη τῆς Ευρώπης μαζί με τὴ Ρώμη. ἘΣ
ίναι γνωστή σε όλο τον κόσμο για τα ιστοριχά τῆς μνημεία που διασώϑγηραν,έστω
χκαι μερικώς, στο πέρασμα τῶν αἰώνων. Ἐπίνειο τῆς ἱιστορικής πόλης εἰναι το λι-
μάνι του Πειραιά. Πολιούχος τῆς Πόλης των Αϑηνών εἰναι ο Ἅγιος Διονύσιος ο
ἈΑρεοπαγίτης.


## Using Tika

### Starting it up

In [32]:
import tika
import requests
from tika import parser

# Start running the tika service
tika.initVM()

### Doing your parsing

There are two ways to do it!

**Right from the web**

```python
response = requests.get(...)
results = parser.from_buffer(response.content)
```

**From a downloaded file**

```python
results = parser.from_file(filename)
```

Note if you want to do **non-English OCR**, you need to change things up a bit. The one below for Greek. See what your tesseract supports with `tesseract --list-langs`

```python
headers = {
    "X-Tika-OCRLanguage": "grc"
}

results = parser.from_buffer(response.content, headers=headers)
```

## Examples

### .pdf example

The first time it will be very slow, as it's... downloading Tika again, I think?

In [33]:
response = requests.get('http://www.data.gov.gr/dataset/a7b2fb4b-e6cb-452f-9e5f-4ebbc9292fa8/resource/04d455da-f644-430e-8b2b-57853358fb91/download/2019---.pdf')
results = parser.from_buffer(response.content)

In [34]:
results.keys()

dict_keys(['status', 'content', 'metadata'])

In [35]:
results['status']

200

In [52]:
# Only showing the first 500 chars because there are SO MANY
results['content'][:1000]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\tΚΕΝΤΡΟ ΜΕΛΕΤΩΝ ΑΣΦΑΛΕΙΑΣ   \n\n\t\n\tCENTER FOR SECURITY STUDIES\n\n\n\nΠΡΟΓΡΑΜΜΑΤΑ ΕΚΠΑΙΔΕΥΣΗΣ\n\nΔιοργάνωση Πιλοτικού Προγράμματος Εκπαίδευσης με τίτλο: «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)» \nΔημοσιεύθηκε : 13 Αυγούστου 2018 \nΤο Κέντρο Μελετών Ασφάλειας (ΚΕ.ΜΕ.Α.) του Υπουργείου Εσωτερικών, διοργανώνει στις εγκαταστάσεις του (Λ. Μεσογείων 96, Πρώην Σ.Α.Ε.Α.) πιλοτικό εκπαιδευτικό πρόγραμμα με τίτλο «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)». Το πρόγραμμα που τελεί υπό την αιγίδα του Ευρωπαϊκού Κολλεγίου Ασφάλειας και Άμυνας (ESDC) θα πραγματοποιηθεί το διάστημα από 8-12 Οκτωβρίου 2018.\nΣτόχος του προγράμματος είναι η κατανόηση του πολύπλευρου φαινομένου της μετανάστευσης με έμφαση στην περιοχή της Μέσης Ανατολής και της Βόρειας Αφρικής (ΜΕΝΑ) και η εξέταση του τρόπου με τον οπ

In [49]:
# Only showing the first 10000 chars
print(results['content'][:10000].strip())

ΚΕΝΤΡΟ ΜΕΛΕΤΩΝ ΑΣΦΑΛΕΙΑΣ   

	
	CENTER FOR SECURITY STUDIES



ΠΡΟΓΡΑΜΜΑΤΑ ΕΚΠΑΙΔΕΥΣΗΣ

Διοργάνωση Πιλοτικού Προγράμματος Εκπαίδευσης με τίτλο: «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)» 
Δημοσιεύθηκε : 13 Αυγούστου 2018 
Το Κέντρο Μελετών Ασφάλειας (ΚΕ.ΜΕ.Α.) του Υπουργείου Εσωτερικών, διοργανώνει στις εγκαταστάσεις του (Λ. Μεσογείων 96, Πρώην Σ.Α.Ε.Α.) πιλοτικό εκπαιδευτικό πρόγραμμα με τίτλο «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)». Το πρόγραμμα που τελεί υπό την αιγίδα του Ευρωπαϊκού Κολλεγίου Ασφάλειας και Άμυνας (ESDC) θα πραγματοποιηθεί το διάστημα από 8-12 Οκτωβρίου 2018.
Στόχος του προγράμματος είναι η κατανόηση του πολύπλευρου φαινομένου της μετανάστευσης με έμφαση στην περιοχή της Μέσης Ανατολής και της Βόρειας Αφρικής (ΜΕΝΑ) και η εξέταση του τρόπου με τον οποίο οι ευρωπαϊκές πολιτικές για την μετανάστευση εντάσσονται στο ευρύτερο πλαίσιο των παρεμβάσεων της Παγκόσμιας Στρατηγική

### .doc example

In [46]:
response = requests.get('http://www.data.gov.gr/dataset/3e5cce22-026c-4f3a-9581-95a574f83ef2/resource/0fa049fa-e821-4971-a5b4-9559ffed99a3/download/.docx')
results = parser.from_buffer(response.content)
# Only doing the first 5000 chars
print(results['content'][:5000].strip())

ΚΕΝΤΡΟ ΜΕΛΕΤΩΝ ΑΣΦΑΛΕΙΑΣ   

	
	CENTER FOR SECURITY STUDIES



ΠΡΟΓΡΑΜΜΑΤΑ ΕΚΠΑΙΔΕΥΣΗΣ

Διοργάνωση Πιλοτικού Προγράμματος Εκπαίδευσης με τίτλο: «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)» 
Δημοσιεύθηκε : 13 Αυγούστου 2018 
Το Κέντρο Μελετών Ασφάλειας (ΚΕ.ΜΕ.Α.) του Υπουργείου Εσωτερικών, διοργανώνει στις εγκαταστάσεις του (Λ. Μεσογείων 96, Πρώην Σ.Α.Ε.Α.) πιλοτικό εκπαιδευτικό πρόγραμμα με τίτλο «Μεταναστευτικές Ροές, Διαχείριση Συνόρων και Κοινή Πολιτική Άμυνας και Ασφάλειας (ΚΠΑΑ)». Το πρόγραμμα που τελεί υπό την αιγίδα του Ευρωπαϊκού Κολλεγίου Ασφάλειας και Άμυνας (ESDC) θα πραγματοποιηθεί το διάστημα από 8-12 Οκτωβρίου 2018.
Στόχος του προγράμματος είναι η κατανόηση του πολύπλευρου φαινομένου της μετανάστευσης με έμφαση στην περιοχή της Μέσης Ανατολής και της Βόρειας Αφρικής (ΜΕΝΑ) και η εξέταση του τρόπου με τον οποίο οι ευρωπαϊκές πολιτικές για την μετανάστευση εντάσσονται στο ευρύτερο πλαίσιο των παρεμβάσεων της Παγκόσμιας Στρατηγική

### OCR .png example

It should work the same with a PDF instead of a png, but I didn't find any non-English PDFs laying around.

In [29]:
headers = {
    "X-Tika-OCRLanguage": "grc"
}

response = requests.get('https://i.stack.imgur.com/HUy7V.png')
results = parser.from_buffer(response.content, headers=headers)
results['status']

200

In [31]:
print(results['content'].strip())

Η Αϑήνα (Αϑῆναι στὰ ἀρχαία ελληνικά και τὴν καϑαρεύουσα) εἶναι ἡ πρω-
τεύουσα τῆς Βλλάδας. Ἐπίσης είναι ἡ ἔδρα τῆς Περιφέρειας Αττικής. Βρίσχεται
στὴ Στερεά (Κεντρική) Ἑλλάδα και αποτελεί εύρωστο οιχονομιχό, πολιτιστικό χαι
διοικητικό κέντρο τῆς χώρας. Πήρε το ὀνομά τῆς από τὴ ϑεά Αϑηνά που ἦταν χαι
ἡ προστάτιδά της. Η Αϑήνα σήμερα εἰναι μία σύγχρονη πόλη αλλά χαι διάσημη,
χκαϑώς στὴν αρχαιότητα ἦταν πανίσχυρη πόλη-χράτος και σημαντικότατο χέντρο
πολιτισμού. ϑεωρείται ἡ ἱιστορικότερη πόλη τῆς Ευρώπης μαζί με τὴ Ρώμη. ἘΣ
ίναι γνωστή σε όλο τον κόσμο για τα ιστοριχά τῆς μνημεία που διασώϑγηραν,έστω
χκαι μερικώς, στο πέρασμα τῶν αἰώνων. Ἐπίνειο τῆς ἱιστορικής πόλης εἰναι το λι-
μάνι του Πειραιά. Πολιούχος τῆς Πόλης των Αϑηνών εἰναι ο Ἅγιος Διονύσιος ο
ἈΑρεοπαγίτης.


### Using local files

In [43]:
# Save the file locally
!curl -O https://i.stack.imgur.com/HUy7V.png

# Now let's process it using parser.from_file
headers = {
   "X-Tika-OCRLanguage": "grc"
}
results = parser.from_file('HUy7V.png', headers=headers)
results['content'].strip()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  216k  100  216k    0     0   965k      0 --:--:-- --:--:-- --:--:--  968k


'Η Αϑήνα (Αϑῆναι στὰ ἀρχαία ελληνικά και τὴν καϑαρεύουσα) εἶναι ἡ πρω-\nτεύουσα τῆς Βλλάδας. Ἐπίσης είναι ἡ ἔδρα τῆς Περιφέρειας Αττικής. Βρίσχεται\nστὴ Στερεά (Κεντρική) Ἑλλάδα και αποτελεί εύρωστο οιχονομιχό, πολιτιστικό χαι\nδιοικητικό κέντρο τῆς χώρας. Πήρε το ὀνομά τῆς από τὴ ϑεά Αϑηνά που ἦταν χαι\nἡ προστάτιδά της. Η Αϑήνα σήμερα εἰναι μία σύγχρονη πόλη αλλά χαι διάσημη,\nχκαϑώς στὴν αρχαιότητα ἦταν πανίσχυρη πόλη-χράτος και σημαντικότατο χέντρο\nπολιτισμού. ϑεωρείται ἡ ἱιστορικότερη πόλη τῆς Ευρώπης μαζί με τὴ Ρώμη. ἘΣ\nίναι γνωστή σε όλο τον κόσμο για τα ιστοριχά τῆς μνημεία που διασώϑγηραν,έστω\nχκαι μερικώς, στο πέρασμα τῶν αἰώνων. Ἐπίνειο τῆς ἱιστορικής πόλης εἰναι το λι-\nμάνι του Πειραιά. Πολιούχος τῆς Πόλης των Αϑηνών εἰναι ο Ἅγιος Διονύσιος ο\nἈΑρεοπαγίτης.'